In [314]:
import numpy as np
from decimal import *

def convert(tu):
    origin = [0,0,0,0,0,0,0,0,0,0,0,0]
    for i in tu:
        origin[i%12]=1
    return tuple(origin)

# chord = {
#     convert((0,4,7)):'Major',convert((0,3,7)):'minor',
#     convert((0,4,8)):'aug',convert((0,3,6)):'dim',
#     convert((0,4,7,11)):'Major7',convert((0,4,7,10)):'dom7',
#     convert((0,3,7,10)):'minor7',convert((0,3,6,10)):'m7flat5',
#     convert((0,2,7)):'2',convert((0,5,7)):'4',
#     convert((0,2,4,7,10)):'9'
# }
chord = {
    convert((0,4,7)):'C',convert((2,5,9)):'Dm',convert((4,7,11)):'Em',
    convert((0,5,9)):'F',convert((2,7,11)):'G',convert((0,4,9)):'Am',
    convert((2,5,10)):'Bb',convert((2,5,11)):'Bdim'
}

chord_np = np.array(chord.keys())

key = {
    0:'C',2:'Dm',4:'Em',5:'F',7:'G',9:'Am',11:'Bdim'
} # pitch%12

time = {
    'measure':None,
    'whole':Decimal(1),
    'half':Decimal(1)/2,
    'quarter':Decimal(1)/4,
    'eighth':Decimal(1)/8,
    '16th':Decimal(1)/16,
    '32nd':Decimal(1)/32,
    '64th':Decimal(1)/64,
    '128th':Decimal(1)/128
}

def ifDots(tag):
    if not tag.find('dots'):
        multi = 1
    else:
        dots = tag.find('dots').text
        if dots == '1':
            multi = 1.5
        elif dots == '2':
            multi = 1.75
        elif dots == '3':
            multi = 1.875
    return Decimal(multi)

# 拍點音符紀錄器 for 和弦判斷
- 附點< dots >, 多連音< Tuplet >, 各種拍子、變拍< TimeSig >已處理
- dic 為已取出之(每個staff的)拍子和音高


In [315]:
from bs4 import BeautifulSoup as bs
import sys

addr = 'C:/Users/BigData/Desktop/Armitage Shanks.mscx'
f = open(addr,'r')
music = bs(f.read(),'xml')
f.close()
staff1TimeSig = music.select_one('Score > Staff:nth-of-type(1) TimeSig')
sigN = Decimal(staff1TimeSig.find('sigN').text)
sigD = Decimal(staff1TimeSig.find('sigD').text)
time['measure'] = sigN/sigD
l = []
dic = {}
nonpitched = set(map(lambda x: x.get('id') if x.StaffType.get('group') != 'pitched' else 0,music.select('Part > Staff')))
for staff in set(music.select('Score > Staff'))-nonpitched:
    time_acc = 0
    tupletID = ''
    tupletRatio = 1
    for tag in staff.find_all(['Rest','Chord','Tuplet','TimeSig']):

        #Additional <track> in every measure is ignored
        if (tag.name == 'Chord' or tag.name == 'Rest') and not tag.find('track'):
            multi = ifDots(tag)
            if tag.find('Tuplet') and tag.find('Tuplet').text == tupletID:
                multi *= tupletRatio
            duTime = Decimal(time[tag.select('durationType')[0].text])
            time_acc += duTime * multi
            if tag.name == 'Rest':
                l.append((time_acc, 0))
            else:
                pitches = tag.select('pitch')
                
                #目前音高為手動修改
                addlist = [time_acc] + [int(a.text) for a in pitches]
                l.append(tuple(addlist))
        
        #Only take <Tuplet> with id to avoid children
        #<Tuplet> inside <Chord> or <Rest>
        elif tag.name == 'Tuplet' and tag.get('id'):
            tupletID = tag['id']
            normalNotes = Decimal(tag.find('normalNotes').text)
            actualNotes = Decimal(tag.find('actualNotes').text)
            tupletRatio = normalNotes / actualNotes
        
        elif tag.name == 'TimeSig':
            sigN = Decimal(tag.find('sigN').text)
            sigD = Decimal(tag.find('sigD').text)
            if sigN/sigD != time['measure']:
                time['measure'] = sigN/sigD
    dic.update({'staff_'+staff.get('id') : l})
    l = []

# 開始判斷和弦
只看拍點(目前針對4/4拍，每兩拍判斷一次拍點)
優先順序：
- 整個staff扣除休止符後有50%比率同時演奏兩個音以上，紀錄為和弦聲部
- 若沒有任何和弦聲部，跳過此首歌，在後面處理
- 判斷拍點上所有和弦聲部累加的音所屬和弦
- 若當時沒有任何和弦演奏，用BASS因當作基音判斷
- 有少數情況是大多數和弦出現在後半拍(或下一拍)，若發現此情形則改用後半拍為拍點判定(或下一拍)

In [316]:
pitch_count = 0
percent = 0.5

def have_chord(li):
    return float(len([a for a in li if len(a)>2]))
def have_note(li):
    return float(len([a for a in li if a[1] != 0]))
dominant_percent_staff_list = filter(lambda x: have_chord(dic[x])/have_note(dic[x]) >= percent,dic)

print dominant_percent_staff_list

[u'staff_3']


### 處理可能的例外狀況

In [317]:
shifting_staff_list = []
det_mult = 2

for staff in dominant_percent_staff_list:
    chord_count = 0
    chord_half_count = 0
    chord_one_count = 0
    count = 0
    begin = 0        
    while(count < dic[staff][-1][0]):
        while(count < dic[staff][begin][0]):
            if dic[staff][begin][1] != 0:
                if count*8 % 4 == 0:
                    chord_count += 1
                elif count*8 % 4 == 1:
                    chord_half_count += 1
                elif count*8 % 4 == 2:
                    chord_one_count += 1 
            count += 1.0/8
        begin += 1
    if chord_count==0:   
        if chord_one_count>=chord_half_count:
            shifting_staff_list.append((staff,1.0/4))
        elif chord_half_count>chord_one_count:
            shifting_staff_list.append((staff,1.0/8))
    else:
        if float(chord_one_count)/chord_count>det_mult:
            shifting_staff_list.append((staff,1.0/4))
        elif float(chord_half_count)/chord_count>det_mult:
            shifting_staff_list.append((staff,1.0/8))
        else:
            shifting_staff_list.append((staff,0))

In [318]:
for staff,shift in shifting_staff_list:
    print staff,shift

staff_3 0


In [319]:
import numpy as np
import math
def nearest(nparray, pt): # 歐氏距離
    n = [math.sqrt(sum((i-pt)**2)) for i in nparray]
    mindis = min(n)
    idx = n.index(mindis)
    return nparray[idx]


## 用累加的方式判斷拍點
- 每兩拍判斷一次
- 只判斷拍點
- 用set收集得到的pitch，用聯集表示音高相加
- 休止符 = 空集合

In [320]:
chord_dic = {}
for staff,shift in shifting_staff_list:
    chord_gogo = []
    count = shift
    begin = 0        
    while(count < dic[staff][-1][0]):
        while(count < dic[staff][begin][0]):
            if dic[staff][begin][1] == 0:
                chord_gogo.append(set())
            else:
                now = [pit%12 for pit in dic[staff][begin][1:]]
                if len(now)>1:
                    chord_gogo.append(set(now))
                else:
                    chord_gogo.append(set())
            count += 1.0/2 
        begin += 1
    chord_dic.update({staff : chord_gogo})

In [321]:
# for staff in chord_dic:
#     print staff, chord_dic[staff]
#     print '-----------------------------'
all_chord =  tuple(chord_dic.values())
combined = map(lambda *args: reduce(lambda x,y: x|y,args),*all_chord)

## 用距離判定最靠近的和弦

In [323]:
final_chord = []
for a in combined:
    if list(a) == []:
        final_chord.append(0)
    else:
        closest = nearest(chord_np,convert(tuple(a)))
        final_chord.append(chord[tuple(closest)])
print final_chord

[0, 0, 0, 0, 0, 0, 0, 0, 'Bb', 'Bb', 'Bb', 'Bb', 'Bb', 'Bb', 'Bb', 'Bb', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'Bb', 'Bb', 'Bb', 'Bb', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'Bb', 'Bb', 'Bb', 'Bb', 'F', 'F', 'F', 'F', 'Bb', 'Bb', 'F', 'F', 'F', 'F', 'F', 'F', 'Bb', 'Bb', 'Bb', 'Bb', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'Bb', 'Bb', 'Bb', 'Bb', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'Bb', 'Bb', 'Bb', 'Bb', 'F', 'F', 'F', 'F', 'Bb', 'Bb', 'F', 'F', 'F', 'F', 'F', 'F', 'Bb', 'Bb', 'Bb', 'Bb', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'Bb', 'Bb', 'Bb', 'Bb', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'Bb', 0, 0, 0, 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'Bb', 'Bb', 'Bb', 'Bb', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'Bb', 'Bb', 'Bb', 'Bb', 'F', 'F', 'F', 'F', 'Bb', 'Bb', 'F', 'F', 'F', 'F', 'F', 'F', 'Bb', 'Bb', 'Bb', 'Bb', 'F', '

In [324]:
c = []
for i in range(1,len(final_chord)+1):
    if i%8 == 4:
        c.append(final_chord[i-1])
        print c
        c = []
    else:
        c.append(final_chord[i-1])


[0, 0, 0, 0]
[0, 0, 0, 0, 'Bb', 'Bb', 'Bb', 'Bb']
['Bb', 'Bb', 'Bb', 'Bb', 'F', 'F', 'F', 'F']
['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F']
['Bb', 'Bb', 'Bb', 'Bb', 'F', 'F', 'F', 'F']
['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F']
['Bb', 'Bb', 'Bb', 'Bb', 'F', 'F', 'F', 'F']
['Bb', 'Bb', 'F', 'F', 'F', 'F', 'F', 'F']
['Bb', 'Bb', 'Bb', 'Bb', 'F', 'F', 'F', 'F']
['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F']
['Bb', 'Bb', 'Bb', 'Bb', 'F', 'F', 'F', 'F']
['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F']
['Bb', 'Bb', 'Bb', 'Bb', 'F', 'F', 'F', 'F']
['Bb', 'Bb', 'F', 'F', 'F', 'F', 'F', 'F']
['Bb', 'Bb', 'Bb', 'Bb', 'F', 'F', 'F', 'F']
['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F']
['Bb', 'Bb', 'Bb', 'Bb', 'F', 'F', 'F', 'F']
['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F']
['Bb', 0, 0, 0, 'F', 'F', 'F', 'F']
['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F']
['Bb', 'Bb', 'Bb', 'Bb', 'F', 'F', 'F', 'F']
['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F']
['Bb', 'Bb', 'Bb', 'Bb', 'F', 'F', 'F', 'F']
['Bb', 'Bb', 'F', 'F', 'F', 'F', 'F', 'F']
['Bb', 

In [48]:
l1 = [2,4,5]
l2 = [1,2,3]
l3 = [3,3,4]
a = (l1,l2,l3)
print map(lambda *args: sum(args),*a)

[6, 9, 12]


In [57]:
s1 = set([2,4,5])
s2 = set([1,2,3])
s3 = set([3,3,4])
a = (s1,s2,s3)
print reduce(lambda x,y: x|y,a)
# print map(lambda *args: sum(args),*a)

set([1, 2, 3, 4, 5])


In [70]:
chord_gogo = []
count = 0
begin = 0
while(count<l2[-1][0]):
    while(count < l2[begin][0]):
        count += 1.0/2
        if l2[begin][1] == 0:
            chord_gogo.append(0)
        else:
            chord_gogo.append(key[l2[begin][1]%12])
    begin += 1

[5, 1, 3]
